In [ ]:
@log_errors(ect_logger)
async def get_filtered_earnings(
    self,
    refinitiv_ids: List[str],
    year: Optional[int] = None,
    quarter: Optional[str] = None
) -> List[EarningTranscript]:
    """
    Fetch earnings call transcripts filtered by year and/or quarter using metadata fields
    
    Args:
        refinitiv_ids: List of Refinitiv company IDs
        year: Optional year filter (e.g., 2024)
        quarter: Optional quarter filter (e.g., "Q1", "Q2", "Q3", "Q4")
    """
    transcripts = []
    try:
        earnings_metadata_list = await self.get_earnings_metadata_by_refinitiv_id(
            refinitiv_ids
        )
        ect_logger.info(
            f"Earnings metadata fetched for companies: {format(refinitiv_ids)}"
        )
        
        for refinitiv_id in refinitiv_ids:
            # Filter metadata for current company
            earnings_metadata = [
                metadata 
                for metadata in earnings_metadata_list 
                if metadata.companyId_s == refinitiv_id
            ]
            
            if not earnings_metadata:
                ect_logger.warning(
                    f"No earnings metadata found for company: {refinitiv_id}"
                )
                continue
                
            # Apply year/quarter filters using metadata fields
            filtered_metadata = []
            for metadata in earnings_metadata:
                # Check year if specified
                if year:
                    if not metadata.year_s or metadata.year_s != str(year):
                        continue
                        
                # Check quarter if specified
                if quarter:
                    if not metadata.quarter_s or metadata.quarter_s != quarter:
                        continue
                        
                filtered_metadata.append(metadata)
                    
            if not filtered_metadata:
                ect_logger.warning(
                    f"No earnings found for {year or 'any year'} {quarter or 'any quarter'} "
                    f"for company: {refinitiv_id}"
                )
                continue
                
            # Get latest from filtered results
            latest_metadata = self.get_latest_earnings_metadata(filtered_metadata)
            if not latest_metadata or not latest_metadata.objecturl:
                ect_logger.warning(
                    f"No earnings url found for filtered metadata of company: {refinitiv_id}"
                )
                continue
                
            ect_logger.info(f"{format(latest_metadata)}")
            headers = {"Cookie": f"GSSSO={self.gssso_token}"}
            
            # Fetch transcript
            async with aiohttp.ClientSession() as session:
                async with session.get(
                    latest_metadata.objecturl,
                    headers=headers
                ) as response:
                    if response.status != 200:
                        ect_logger.error(
                            f"Failed to fetch content for company {refinitiv_id}, status: {response.status}"
                        )
                        continue
                        
                    transcripts.append(
                        EarningTranscript(
                            transcript=await response.text(),
                            metadata=latest_metadata,
                        )
                    )
                    
    except ContentFetchError as e:
        ect_logger.error(f"Content fetch error: {e}")
        raise
    except Exception as e:
        ect_logger.error(f"Unexpected error: {e}")
        raise ContentFetchError(
            "Failed to fetch the filtered earning transcripts due to an unexpected error"
        )
        
    return transcripts